In [1]:
import os
import math

# ---------------------------------------------------------
# 1. CONFIGURATION & MATH ENGINE
# ---------------------------------------------------------

BASE_DIR = "optimization_scenarios"

# Explicit Bin Dimensions (Must match Validator's 'Real' calculation W*D*H)
LOC_SPECS = {
    "A1": (160.0, 600.0, 130.0), # Vol: 12,480,000 mm3
    "A2": (240.0, 790.0, 220.0), # Vol: 41,712,000 mm3
    "A3": (410.0, 850.0, 260.0)  # Vol: 90,610,000 mm3
}

def calculate_allocation_row(loc_code, loc_type, item_id, item_dims, qty):
    """
    Generates a mathematically valid CSV row.
    Strictly uses W*D*H for both Item and Location to ensure 
    Calc Utilization matches Stated Utilization.
    """
    # 1. Get Dimensions
    l_bin, d_bin, h_bin = LOC_SPECS[loc_type]
    l_item, w_item, h_item = item_dims 
    
    # 2. Calculate Capacity (Grid)
    # Floor division to get integer slots
    grid_x = max(1, int(l_bin // l_item))
    grid_y = max(1, int(d_bin // w_item))
    grid_z = max(1, int(h_bin // h_item))
    
    max_units = grid_x * grid_y * grid_z

    # 3. Calculate Volumes (mm3)
    # Validator calculates "Real" volume as W*D*H
    loc_vol_mm3 = l_bin * d_bin * h_bin
    loc_vol_m3 = loc_vol_mm3 / 1e9

    item_vol_mm3 = l_item * w_item * h_item
    stored_vol_mm3 = item_vol_mm3 * qty
    stored_vol_m3 = stored_vol_mm3 / 1e9
    
    # 4. Calculate Utilization %
    # Formula: (Total Item Vol / Total Bin Vol) * 100
    util_pct = (stored_vol_mm3 / loc_vol_mm3) * 100
    
    # 5. Layer Logic
    items_per_layer = grid_x * grid_y
    if items_per_layer > 0:
        full_layers = int(qty // items_per_layer)
        partial_units = int(qty % items_per_layer)
    else:
        full_layers = 0
        partial_units = qty

    # 6. Orientation (Pass through dimensions)
    orient_x, orient_y, orient_z = l_item, w_item, h_item

    # Format string (Note: High precision for util_pct to avoid rounding mismatches)
    return f"{loc_code},{loc_type},{item_id},{int(qty)},{max_units},{grid_x},{grid_y},{grid_z},{full_layers},{partial_units},{orient_x:.1f},{orient_y:.1f},{orient_z:.1f},{loc_vol_mm3:.1f},{loc_vol_m3:.4f},{stored_vol_m3:.4f},{util_pct:.4f}"

# ---------------------------------------------------------
# 2. SCENARIO DATA DEFINITIONS
# ---------------------------------------------------------

scenarios = [
    {
        "folder": "Scenario_1_Weight_Safety",
        "readme": """SCENARIO 1: WEIGHT SAFETY PROTOCOL
----------------------------------
OBJECTIVE:
Verify Hard Constraint: Items > 15kg must not be stored above Z = 1500mm.

INPUT STATE:
1. '99-HEAVY' (20kg) is at A1-00016 (Z=1950mm). VIOLATION.
2. '11-LIGHT' (0.5kg) is at A1-00001 (Z=0mm). Safe.

EXPECTED OUTPUT:
- 99-HEAVY moves down.
""",
        "parts": [
            # Dims fit cleanly in A1 (160x600x130)
            # Item: 150x150x100
            ("99-HEAVY", "HEAVY_PART", 150.0, 150.0, 100.0, 20.0, 1, 4, 1),
            ("11-LIGHT", "LIGHT_PART", 150.0, 150.0, 100.0, 0.5, 1, 4, 1)
        ],
        "allocs": [
            ("A1-00016", "A1", "99-HEAVY", 4),
            ("A1-00001", "A1", "11-LIGHT", 4)
        ]
    },
    {
        "folder": "Scenario_2_Storage_Utilization",
        "readme": """SCENARIO 2: STORAGE UTILIZATION
----------------------------------
OBJECTIVE:
Prioritize density. Move small items out of large bins.

INPUT STATE:
1. '22-STD' (80mm cube) is in A2-00001 (Large Bin). Low Util.
2. '33-BIG' (160x600x130) is in A1-00005 (Small Bin). High Util (100%).
3. A1-00001 is Empty.

EXPECTED OUTPUT:
- '22-STD' moves from A2 to A1-00001.
""",
        "parts": [
            # 22-STD: 80x80x80 (Vol 512k). Fits A2 (240x790x220).
            ("22-STD", "STD_PART", 80.0, 80.0, 80.0, 0.2, 1, 10, 5),
            # 33-BIG: Exact dimensions of A1 (160x600x130)
            ("33-BIG", "LARGE_PART", 160.0, 600.0, 130.0, 2.0, 1, 1, 5)
        ],
        "allocs": [
            ("A2-00001", "A2", "22-STD", 10),
            ("A1-00005", "A1", "33-BIG", 1)
        ]
    },
    {
        "folder": "Scenario_3_Picking_Efficiency",
        "readme": """SCENARIO 3: PICKING EFFICIENCY
----------------------------------
OBJECTIVE:
Class A items (High Demand) -> Fast/Ergo Zone.

INPUT STATE:
1. '44-FAST' (Demand 500) is at A2-00150 (Bad Spot).
2. '55-SLOW' (Demand 1) is at A1-00009 (Good Spot).

EXPECTED OUTPUT:
- Swap: 44-FAST moves to Good Spot.
""",
        "parts": [
            # Dims: 100x100x100 (1M mm3). Fits both A1 and A2 easily.
            ("44-FAST", "HIGH_DEMAND", 100.0, 100.0, 100.0, 1.0, 1, 5, 500),
            ("55-SLOW", "LOW_DEMAND", 100.0, 100.0, 100.0, 1.0, 1, 5, 1)
        ],
        "allocs": [
            ("A2-00150", "A2", "44-FAST", 5),
            ("A1-00009", "A1", "55-SLOW", 5)
        ]
    },
    {
        "folder": "Scenario_4_Balancing_Conflicts",
        "readme": """SCENARIO 4: BALANCING CONFLICTS
----------------------------------
OBJECTIVE:
Safety > Efficiency.

INPUT STATE:
1. '66-HVY-FST' (20kg) at A2-00100 (High Z). VIOLATION.
2. '77-LGT-FST' (1kg) at A2-00014 (Low Z). Safe.

EXPECTED OUTPUT:
- Heavy moves to Safe (Low Z).
""",
        "parts": [
            # 100x100x100 fits easily
            ("66-HVY-FST", "HEAVY_DEMAND", 100.0, 100.0, 100.0, 20.0, 1, 5, 200),
            ("77-LGT-FST", "LIGHT_DEMAND", 100.0, 100.0, 100.0, 1.0, 1, 5, 200)
        ],
        "allocs": [
            ("A2-00100", "A2", "66-HVY-FST", 5),
            ("A2-00014", "A2", "77-LGT-FST", 5)
        ]
    }
]

# ---------------------------------------------------------
# 3. GENERATION SCRIPT
# ---------------------------------------------------------

def generate_files():
    if not os.path.exists(BASE_DIR):
        os.makedirs(BASE_DIR)

    alloc_header = "loc_inst_code,LOCATION_TYPE,ITEM_ID,QTY_ALLOCATED,MAX_UNITS,GRID_X,GRID_Y,GRID_Z,FULL_LAYERS,PARTIAL_UNITS,ORIENT_X_MM,ORIENT_Y_MM,ORIENT_Z_MM,LOCATION_VOL_MM3,LOCATION_VOL_M3,STORED_VOL_M3,UTILIZATION_PCT"
    parts_header = "ITEM_ID;ITEM_DESC;LEN_MM;WID_MM;DEP_MM;WT_KG;QTY_PER_BOX;BOXES_ON_HAND;DEMAND"

    for scen in scenarios:
        path = os.path.join(BASE_DIR, scen["folder"])
        if not os.path.exists(path):
            os.makedirs(path)

        # 1. Write README
        with open(os.path.join(path, "README.txt"), "w") as f:
            f.write(scen["readme"])

        # 2. Write Parts CSV
        with open(os.path.join(path, "synthetic_parts_generated.csv"), "w") as f:
            f.write(parts_header + "\n")
            for p in scen["parts"]:
                # Consistent formatting
                line = f"{p[0]};{p[1]};{p[2]:.1f};{p[3]:.1f};{p[4]:.1f};{p[5]:.2f};{int(p[6])};{int(p[7])};{int(p[8])}"
                f.write(line + "\n")

        # 3. Write Allocations CSV
        with open(os.path.join(path, "allocations.csv"), "w") as f:
            f.write(alloc_header + "\n")
            for alloc in scen["allocs"]:
                loc_code, loc_type, item_id, qty = alloc
                
                # Retrieve item dimensions
                item = next((i for i in scen["parts"] if i[0] == item_id), None)
                if item:
                    dims = (item[2], item[3], item[4]) # L, W, D
                    row_str = calculate_allocation_row(loc_code, loc_type, item_id, dims, qty)
                    f.write(row_str + "\n")
                else:
                    print(f"ERROR: Item {item_id} not found.")

        print(f"Generated {scen['folder']}")

    print("\nGeneration Complete.")

if __name__ == "__main__":
    generate_files()

Generated Scenario_1_Weight_Safety
Generated Scenario_2_Storage_Utilization
Generated Scenario_3_Picking_Efficiency
Generated Scenario_4_Balancing_Conflicts

Generation Complete.
